In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import hypot, col, max, sum
from pyspark.sql.functions import countDistinct

spark = SparkSession.builder.getOrCreate()

# 1. Найти велосипед с максимальным временем пробега.

In [55]:
# Загрузка данных из CSV в DataFrame
tripData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trip.csv")

In [56]:
# Найти максимальное значение duration
max_duration = tripData.agg(max(col("duration"))).collect()[0][0]

# Найти все велосипеды с этой максимальной продолжительностью
max_duration_trips = (
    tripData.filter(col("duration") == max_duration)
    .select("bike_id", "duration")
)

max_duration_trips.show()

+-------+--------+
|bike_id|duration|
+-------+--------+
|    535|17270400|
+-------+--------+



# 2. Найти наибольшее геодезическое расстояние между станциями.

In [57]:
# Загрузка данных из CSV в DataFrame
stationData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y')\
.csv("station.csv")

In [58]:
# Создаём пары станций
station_pairs = stationData.alias("s1").crossJoin(stationData.alias("s2")) \
    .filter(col("s1.id") < col("s2.id"))

# Вычисляем расстояние по евклидовой метрике
distance_df = station_pairs.withColumn(
    "distance",
    hypot(col("s2.lat") - col("s1.lat"), col("s2.long") - col("s1.long"))
)

# Находим максимальное расстояние
max_distance = distance_df.orderBy(col("distance").desc()).select(
    col("s1.name").alias("station_1"),
    col("s2.name").alias("station_2"),
    col("distance")
).limit(1)

max_distance.show()

+--------------------+--------------------+------------------+
|           station_1|           station_2|          distance|
+--------------------+--------------------+------------------+
|SJSU - San Salvad...|Embarcadero at Sa...|0.7058482821754397|
+--------------------+--------------------+------------------+



# 3. Найти путь велосипеда с максимальным временем пробега через станции.

In [59]:
# Найдём велосипед с максимальным временем пробега
bike_max_time = (
    tripData.groupBy("bike_id")
    .agg(sum(col("duration")).alias("total_duration"))
    .orderBy(col("total_duration").desc())
    .limit(1)
)

# Получаем его ID
max_bike_id = bike_max_time.collect()[0]["bike_id"]

# Фильтруем поездки этого велосипеда
bike_trips = (
    tripData.filter(col("bike_id") == max_bike_id)
    .orderBy(col("start_date"))
    .select("bike_id", "start_station_name", "end_station_name", "duration")
)

bike_trips.show()

+-------+--------------------+--------------------+--------+
|bike_id|  start_station_name|    end_station_name|duration|
+-------+--------------------+--------------------+--------+
|    535|     Post at Kearney|San Francisco Cal...|    1245|
|    535|San Francisco Cal...|San Francisco Cal...|     423|
|    535|San Francisco Cal...|   Market at Sansome|     842|
|    535|   Market at Sansome|   2nd at South Park|     498|
|    535|     2nd at Townsend|    Davis at Jackson|    1671|
|    535|San Francisco Cit...|Civic Center BART...|     260|
|    535|Civic Center BART...|     Post at Kearney|    1192|
|    535|     Post at Kearney|Embarcadero at Sa...|    1248|
|    535|Embarcadero at Sa...|Washington at Kea...|    1272|
|    535|Washington at Kea...|   Market at Sansome|     398|
|    535|   Market at Sansome|   Market at Sansome|   12476|
|    535|   Market at Sansome|       2nd at Folsom|     582|
|    535|       2nd at Folsom|     2nd at Townsend|     499|
|    535|Temporary Trans

# 4. Найти количество велосипедов в системе

In [60]:
bike_count = tripData.select(countDistinct("bike_id").alias("total_bikes"))

bike_count.show()

+-----------+
|total_bikes|
+-----------+
|        700|
+-----------+



# 5. Найти пользователей потративших на поездки более 3 часов.

In [61]:
# Группируем по 'bike_id' и суммируем время всех поездок для каждого пользователя
users_trip_time = (
    tripData.groupBy("zip_code")
    .agg(sum(col("duration")).alias("total_duration"))
)

# Фильтруем пользователей, чье суммарное время больше 3 часов (10800с)
long_trip_users = users_trip_time.filter(col("total_duration") > 10800)

long_trip_users.show()

+--------+--------------+
|zip_code|total_duration|
+--------+--------------+
|   94102|      19128021|
|   95134|        728023|
|   84606|         95145|
|   80305|        180906|
|   60070|         28919|
|   95519|         30303|
|   43085|         11670|
|   91910|         50488|
|   77339|         13713|
|   48063|         13755|
|   85022|         12682|
|    1090|         20391|
|    2136|         16010|
|   11722|         24331|
|   95138|        155365|
|   94610|       3630628|
|   94404|       3589350|
|   80301|        152189|
|   91326|         65885|
|   90742|         10965|
+--------+--------------+
only showing top 20 rows

